# prologue

### set up notebook and load package

In [1]:
# for notebook plotting
%matplotlib inline 

# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp
from CHIRPS.routines import extend_path

# demo datasets that ship with package. all from UCI unless stated otherwise
# ds.adult_data, ds.adult_samp_data, ds.adult_small_samp_data Large dataset ships with manageable sub samples
# ds.bankmark_data, ds.bankmark_samp_data
# ds.car_data
# ds.cardio_data this is the cardiotocography dataset
# ds.credit_data
# ds.german_data
# ds.lending_data, ds.lending_samp_data, ds.lending_small_samp_data, ds.lending_tiny_samp_data from Kaggle. see datasets_from_source file for links
# ds.nursery_data, ds.nursery_samp_data
# ds.rcdv_data, ds.rcdv_samp_data from US government see datasets_from_source file for links

### common config - can be ommitted if defaults are OK

In [2]:
project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
random_state_splits = 123 # one off for splitting the data into test / train

# data management

### datasets

Several datasets are available as pre-prepared containers that hold the data and some meta-data that is used in the algorithm
Any dataset can be turned into a container by invoking the constructor found in the file structures.py

In [3]:
# datasets might be down-sampled to make them easier to work with
# the full sets are available too
# this is a list of constructors that will be used in the benchmarking
rp.datasets

[<function CHIRPS.datasets.adult_small_samp(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.bankmark_samp(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.car(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.cardio(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.credit(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.german(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.lending_tiny_samp(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.nursery_samp(random_state=123, project_dir=None)>,
 <function CHIRPS.datasets.rcdv_samp(random_state=123, project_dir=None)>]

In [4]:
# example of one dataset
# note: random_state propagates through other functions and is easily updated to allow alternative runs
ds.cardio(random_state=123, project_dir=project_dir)

### standardising train-test splitting
Some methods are not available in Python. We want to maintain the same dataset splits no matter which platform. So, the train test data is split with the one-time random seed and the splits are saved to csv in the project folders.

In [5]:
# writes external files
rp.export_data_splits(datasets=rp.datasets, project_dir=project_dir, random_state_splits=random_state_splits)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

Exported train-test data for 9 datasets.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# Experimental Runs
Loop through datasets, actioning the functions in the package to execute a round of experiments and test evaluations.

## 0. Optional Memory and Computation Cost Management
CHIRPS is time economical but memory intensive to compute for lots of instances at once.


### Parallel processing
Scikit takes care of parallel for the RF construction.
We can parallelise the following:
1. the walk of instances down each tree to collect the paths. The paths for many instances are returned in a single array. This parallelises across trees.
2. building CHIRPS and the final explanation (rule). This is a search optimisation and we can parallelise each instance.

This is expecially effective when running batches. For single instances, set both to false to avoid spinning up the parallel infrastructure.

In [6]:
# control for async processes - each tree walk can be done in its own core
# and so can each explanation (e.g. rule conditions merge by hill-climbing)
# these will default to false if not passed explicitly to the explainer function
# on a multi-core machine there should be a good speed up for large batches
# when the batch_size advantage exceeds the overhead of setting up multi-processing
# timings will be printed to screen so you can see if it helps
forest_walk_async=True
chirps_explanation_async=True

### Preparing unseen data

Again note:
test set has never been "seen" by random forest during training
test set has been only used to assess model (random forest) accuracy - no additional tuning after this
test set has not be involved in generating the explainer

#### Batching
The memory space requirements for all the paths can be reduced by dividing the test set into batches. However this does take longer as there is an overhead to instantiate all the required objects, especially if coupled with parallel processing.
Best compromise could be a small number of larger batches.

In [7]:
# the number of instances can be controlled by
# n_instances - how many instances to explain at one time
# set these larger than the size of the test set and it will simply run the whole test set in one batch. Better do option 2
n_instances = 10000

### 1. Data and Forest prep
Use the random state splits to do a one-off data split.
Fit the RF to training data, using the iterating random state.
Save the performance metrics on the test set for later review.

### 2. Prepare Unseen Data and Predictions
Important to note:
Test set never "seen" by RF during training.
test set not involved in generating the explainer.
Test set used to evaluate model (random forest) accuracy beyond OOBE scores - no additional tuning based on these results.
Test set used to evaluate explanation scores by leave-one-out method removing the specific instance we're explaining.

Important to note:
We will explain predictions directly from the trained RF. Explanation system makes no compromise on model accuracy.

### 3. CHIRPS algorithm
1. Extract tree prediction paths
2. Freqent pattern mining of paths
3. Score and sort mined path segments
4. Merge path segments into one rule

#### CHIRPS 1
Fit a forest_walker object to the dataset and decision forest. This is a wrapper that will extract the paths of all the given instances. Its main method delivers the instance paths for the remaining steps of the algorithm as a new object: a batch_paths_container. It can also report interesting statistics (treating the forest as a set of random tree-structured variables).

#### CHIRPS 2-4
A batch_CHIRPS_container is fitted with the batch_paths_container returned by the forest walker, and with a sample of data. For CHIRPS, we prefer a large sample. The whole training set or other representative sample will do. This is a wrapper object will execute steps 2-4 on all each the instance-paths in the batch_paths_container.

Important to note:
true_divide warnings are OK! It just means that a continuous variable is unbounded on one side i.e. no greater/less than inequality is used in the specific CHIRPS explanation.

Important note: 
Here we are using the training set to create the explainers. We could use a different dataset as long as it is representative of the training set that built the decision forest. Most important that we don't use the dataset that we wish to explain, so never use the test set, for example.

### 4. Evaluating CHIRPS Explanations
Test set has been used to create an explainer *one instance at a time* and the rest of test set was not "seen" during this construction. To score each explainer, we use test set, leaving out the individual instance being explained. The data_split_container (tt) has a convenience funtion for doing this. All the results are saved to csv files in the project directory.

In [8]:
# what to run? Set to True
run_CHIRPS = False
CHIRPS_sensitivity = False

run_CHIRPS_ada = True
CHIRPS_sensada = False

run_Anchors = False
run_defragTrees = True

# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20

# CHIRPS RF default set up
if run_CHIRPS:
    model = 'RandomForest'
    delta = 0.1
    tuning_grid = None
    
    kwargs_default = {'support_paths' : 0.1, 'alpha_paths' : 0.5, 'disc_path_bins' : 4,
                     'score_func' : 1, 'weighting' : 'chisq',
                     'merging_bootstraps' : merging_bootstraps,
                     'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}

# CHIRPS RF Sensitivity set up
if CHIRPS_sensitivity:
    path_ext = 'sensitivity'
    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.1, 0.05], 18), 2)
    weighting = np.repeat(['chisq', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                       'merging_bootstraps' : merging_bootstraps, 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}

# CHIRPS Ada default set up
if run_CHIRPS_ada:
    model='AdaBoost'
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning_grid = {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}

    delta = 0.1
    alpha_paths = 0.0
    score_func = 1
    kwargs_default = {'paths_lengths_threshold' : 5,
                     'support_paths' : 0.01, 'alpha_paths' : alpha_paths,
                     'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
                     'score_func' : score_func, 'weighting' : 'chisq',
                     'merging_bootstraps' : merging_bootstraps,
                     'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}

# CHIRPS Ada Sensitivity set up
if CHIRPS_sensada:
    path_ext = 'sensada1'
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 8)
    support_paths = np.tile([0.05, 0.02, 0.01], 12)
    weighting = np.repeat(['chisq', 'kldiv', 'lodds', 'nothing'], 6)

    kwargs_grid = {k : {'alpha_paths' : alpha_paths, 'disc_path_bins' : dpb, 'score_func' : score_func, 'weighting' : w, 'support_paths' : sp,
                       'merging_bootstraps' : merging_bootstraps, 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta} 
        for k, dpb, w, sp in zip(range(24), disc_path_bins, weighting, support_paths)}

In [9]:
# here will run everything set in the previous panel, in a loop set by each random_state in range.
for random_state in range(123, 124):
    for d_constructor in rp.datasets:
        print('Running experiment for ' + d_constructor.__name__ + ' with random state = ' + str(random_state))
        print()
        # 1. Data and Forest prep
        print('Split data into main train-test and build RF') 
        mydata = d_constructor(random_state=random_state, project_dir=project_dir)
        
        meta_data = mydata.get_meta()
        save_path = meta_data['get_save_path']()
        train_index, test_index = mydata.get_tt_split_idx(random_state=random_state_splits)
        tt = mydata.tt_split(train_index, test_index)
        
        # this will train and score the model
        rf = rp.forest_prep(ds_container=tt,
                    meta_data=meta_data,
                    model=model,
                    tuning_grid=tuning_grid,
                    save_path=save_path)
        
        print()
        if run_CHIRPS or run_CHIRPS_ada:
            if CHIRPS_sensitivity or CHIRPS_sensada:
                for kwg in kwargs_grid:
                    
                    rp.CHIRPS_benchmark(forest=rf, ds_container=tt, meta_data=meta_data, model=model,
                                        n_instances=n_instances,
                                        forest_walk_async=forest_walk_async,
                                        chirps_explanation_async=chirps_explanation_async,
                                        save_path=save_path,
                                        save_sensitivity_path=extend_path(stem=save_path, \
                                                              extensions=[path_ext, \
                                                                'sp_' + str(kwargs_grid[kwg]['support_paths']) + \
                                                                '_ap_' + str(kwargs_grid[kwg]['alpha_paths']) + \
                                                                '_dpb_' + str(kwargs_grid[kwg]['disc_path_bins']) + \
                                                                '_sf_' + str(kwargs_grid[kwg]['score_func']) + \
                                                                '_w_' + str(kwargs_grid[kwg]['weighting']) + '_']),
                                        dataset_name=d_constructor.__name__,
                                        random_state=random_state, **kwargs_grid[kwg])
                    
                    # create a new ds_container as the last one is used up
                    tt = mydata.tt_split(train_index, test_index)
            else:
                rp.CHIRPS_benchmark(forest=rf, ds_container=tt, meta_data=meta_data, model=model,
                                    n_instances=n_instances,
                                    forest_walk_async=forest_walk_async,
                                    chirps_explanation_async=chirps_explanation_async,
                                    save_path=save_path,
                                    dataset_name=d_constructor.__name__,
                                    random_state=random_state, **kwargs_default)
                    
        if run_Anchors:
            # new copy of ds_container (need to reset the row counters)
            tt_anch = mydata.tt_split(train_index, test_index)
            # preprocessing - discretised continuous X matrix has been added and also needs an updated var_dict 
            # plus returning the fitted explainer that holds the data distribution
            tt_anch, anchors_explainer = rp.Anchors_preproc(ds_container=tt_anch,
                                                             meta_data=meta_data)
    
            # re-fitting the random forest to the discretised data and evaluating
            rf = rp.forest_prep(ds_container=tt_anch,
                            meta_data=meta_data,
                            model=model,
                            save_path=save_path,
                            identifier='Anchors')

            rp.Anchors_benchmark(forest=rf, ds_container=tt_anch, meta_data=meta_data,
                                anchors_explainer=anchors_explainer,
                                n_instances=n_instances,
                                save_path=save_path, dataset_name=d_constructor.__name__,
                                random_state=meta_data['random_state'])
        
        if run_defragTrees:
            # create a new copy of tt split, because each one keeps track of which instances it has given out.
            # re-using the top one means different instances are passed
            tt_dfrgtrs = mydata.tt_split(train_index, test_index)
            
            # some dfrgtrs specific parameters
            Kmax = 10
            restart = 20
            maxitr = 100
            dfrgtrs, eval_start_time, defTrees_elapsed_time = rp.defragTrees_prep(ds_container=tt_dfrgtrs,
                                                                                meta_data=meta_data, forest=rf, 
                                                                                Kmax=Kmax, restart=restart, maxitr=maxitr,
                                                                                identifier='defragTrees', save_path=save_path)
            
                        
            rp.defragTrees_benchmark(forest=rf, ds_container=tt_dfrgtrs, meta_data=meta_data, model=model,
                                    dfrgtrs=dfrgtrs, eval_start_time=eval_start_time,
                                    defTrees_elapsed_time=defTrees_elapsed_time,
                                    n_instances=n_instances,
                                    save_path=save_path, dataset_name=d_constructor.__name__,
                                    random_state=random_state)


Running experiment for adult_small_samp with random state = 123

Split data into main train-test and build RF
using previous tuning parameters


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.8508
Best parameters: {'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 1400, 'random_state': 123}


Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 733 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 16.6698 seconds

Running CHIRPS on a batch of 733 instances... (please wait)
{'labels': [0, 1], 'counts': array([31.83357589, 29.6139333 ]), 'p_counts': array([0.51806129, 0.48193871])}
{'labels': [0, 1], 'counts': array([0.09414223, 0.05791423]), 'p_counts': array([0.61912679, 0.38087321])}
{'labels': [0, 1], 'counts': array([36.35807

{'labels': [0, 1], 'counts': array([31.17135483, 30.27615436]), 'p_counts': array([0.50728427, 0.49271573])}
{'labels': [0, 1], 'counts': array([0.01981184, 0.01321661]), 'p_counts': array([0.59984176, 0.40015824])}
{'labels': [0, 1], 'counts': array([34.00418334, 27.44332584]), 'p_counts': array([0.55338587, 0.44661413])}
{'labels': [0, 1], 'counts': array([0.02603098, 0.01541883]), 'p_counts': array([0.62801196, 0.37198804])}
{'labels': [0, 1], 'counts': array([30.65775791, 30.78975128]), 'p_counts': array([0.49892597, 0.50107403])}
{'labels': [0, 1], 'counts': array([0.65634542, 0.1249043 ]), 'p_counts': array([0.84012244, 0.15987756])}
{'labels': [0, 1], 'counts': array([34.17059713, 27.27691205]), 'p_counts': array([0.5560941, 0.4439059])}
{'labels': [0, 1], 'counts': array([1.90569506, 0.01165157]), 'p_counts': array([0.99392308, 0.00607692])}
{'labels': [0, 1], 'counts': array([31.61591976, 29.83158943]), 'p_counts': array([0.51451914, 0.48548086])}
{'labels': [0, 1], 'counts': 

{'labels': [0, 1], 'counts': array([36.46804161, 24.97946757]), 'p_counts': array([0.59348283, 0.40651717])}
{'labels': [0, 1], 'counts': array([0.26575607, 0.01046438]), 'p_counts': array([0.96211583, 0.03788417])}
{'labels': [0, 1], 'counts': array([30.09168418, 31.35582501]), 'p_counts': array([0.48971365, 0.51028635])}
{'labels': [0, 1], 'counts': array([0.01280589, 0.01404371]), 'p_counts': array([0.47694903, 0.52305097])}
{'labels': [0, 1], 'counts': array([37.93791818, 23.50959101]), 'p_counts': array([0.61740368, 0.38259632])}
{'labels': [0, 1], 'counts': array([1.42449852, 0.01273749]), 'p_counts': array([0.99113751, 0.00886249])}
{'labels': [0, 1], 'counts': array([30.81277108, 30.63473811]), 'p_counts': array([0.50144866, 0.49855134])}
{'labels': [0, 1], 'counts': array([0.0122213 , 0.01318015]), 'p_counts': array([0.48112606, 0.51887394])}
{'labels': [0, 1], 'counts': array([33.09164313, 28.35586606]), 'p_counts': array([0.53853514, 0.46146486])}
{'labels': [0, 1], 'counts'

...

{'labels': [0, 1], 'counts': array([0.06401108, 0.00875684]), 'p_counts': array([0.87966073, 0.12033927])}
{'labels': [0, 1], 'counts': array([6.52984131, 6.45766793]), 'p_counts': array([0.50277857, 0.49722143])}
{'labels': [0, 1], 'counts': array([0.01056464, 0.01403791]), 'p_counts': array([0.42941227, 0.57058773])}
{'labels': [0, 1], 'counts': array([6.54907493, 6.43843431]), 'p_counts': array([0.5042595, 0.4957405])}
{'labels': [0, 1], 'counts': array([0.0103864 , 0.00952506]), 'p_counts': array([0.52162914, 0.47837086])}
{'labels': [0, 1], 'counts': array([6.56892717, 6.41858207]), 'p_counts': array([0.50578806, 0.49421194])}
{'labels': [0, 1], 'counts': array([0.01250242, 0.00999969]), 'p_counts': array([0.55561106, 0.44438894])}
{'labels': [0, 1], 'counts': array([7.22861981, 5.75888943]), 'p_counts': array([0.55658246, 0.44341754])}
{'labels': [0, 1], 'counts': array([0.01479472, 0.00800989]), 'p_counts': array([0.64876005, 0.35123995])}
{'labels': [0, 1], 'counts': array([6.1

{'labels': [0, 1], 'counts': array([6.97765422, 6.00985502]), 'p_counts': array([0.53725885, 0.46274115])}
{'labels': [0, 1], 'counts': array([0.01249878, 0.00817219]), 'p_counts': array([0.60465389, 0.39534611])}
{'labels': [0, 1], 'counts': array([6.46042012, 6.52708912]), 'p_counts': array([0.49743334, 0.50256666])}
{'labels': [0, 1], 'counts': array([0.01203765, 0.01217699]), 'p_counts': array([0.49712273, 0.50287727])}
{'labels': [0, 1], 'counts': array([6.96189898, 6.02561026]), 'p_counts': array([0.53604574, 0.46395426])}
{'labels': [0, 1], 'counts': array([0.01369446, 0.00883627]), 'p_counts': array([0.60781258, 0.39218742])}
{'labels': [0, 1], 'counts': array([6.73830028, 6.24920896]), 'p_counts': array([0.5188293, 0.4811707])}
{'labels': [0, 1], 'counts': array([0.01107462, 0.04180053]), 'p_counts': array([0.20944845, 0.79055155])}
{'labels': [0, 1], 'counts': array([7.02096543, 5.96654381]), 'p_counts': array([0.54059368, 0.45940632])}
{'labels': [0, 1], 'counts': array([0.0

{'labels': [0, 1], 'counts': array([6.08523699, 6.90227225]), 'p_counts': array([0.46854534, 0.53145466])}
{'labels': [0, 1], 'counts': array([0.01347554, 0.37596293]), 'p_counts': array([0.03460248, 0.96539752])}
{'labels': [0, 1], 'counts': array([6.51970215, 6.46780709]), 'p_counts': array([0.50199788, 0.49800212])}
{'labels': [0, 1], 'counts': array([0.01434647, 0.01065667]), 'p_counts': array([0.57378672, 0.42621328])}
{'labels': [0, 1], 'counts': array([8.02064093, 4.96686831]), 'p_counts': array([0.61756575, 0.38243425])}
{'labels': [0, 1], 'counts': array([0.0228978 , 0.01010196]), 'p_counts': array([0.69387778, 0.30612222])}
{'labels': [0, 1], 'counts': array([6.68064685, 6.30686239]), 'p_counts': array([0.51439015, 0.48560985])}
{'labels': [0, 1], 'counts': array([0.01099936, 0.00842743]), 'p_counts': array([0.56619536, 0.43380464])}
{'labels': [0, 1], 'counts': array([6.61338392, 6.37412532]), 'p_counts': array([0.5092111, 0.4907889])}
{'labels': [0, 1], 'counts': array([0.0

{'labels': [0, 1], 'counts': array([6.59560678, 6.39190246]), 'p_counts': array([0.50784232, 0.49215768])}
{'labels': [0, 1], 'counts': array([0.0107918 , 0.00947495]), 'p_counts': array([0.5324878, 0.4675122])}
{'labels': [0, 1], 'counts': array([7.39025924, 5.59725   ]), 'p_counts': array([0.56902822, 0.43097178])}
{'labels': [0, 1], 'counts': array([0.03901739, 0.00832992]), 'p_counts': array([0.82406779, 0.17593221])}
{'labels': [0, 1], 'counts': array([7.05938868, 5.92812056]), 'p_counts': array([0.54355216, 0.45644784])}
{'labels': [0, 1], 'counts': array([0.01487325, 0.00928437]), 'p_counts': array([0.61567527, 0.38432473])}
{'labels': [0, 1], 'counts': array([6.51430852, 6.47320072]), 'p_counts': array([0.50158259, 0.49841741])}
{'labels': [0, 1], 'counts': array([0.12623636, 0.01208139]), 'p_counts': array([0.91265481, 0.08734519])}
{'labels': [0, 1], 'counts': array([6.53512289, 6.45238635]), 'p_counts': array([0.50318524, 0.49681476])}
{'labels': [0, 1], 'counts': array([1.6

[Seed 137] TrainingError = 0.38, K = 2
[Seed 138] TrainingError = 0.37, K = 2
[Seed 139] TrainingError = 0.37, K = 2
[Seed 140] TrainingError = 0.37, K = 2
[Seed 141] TrainingError = 0.37, K = 3
[Seed 142] TrainingError = 0.38, K = 3
Optimal Model >> Seed 126, TrainingError = 0.36, K = 3
Fit defragTrees time elapsed: 11.6932 seconds

defragTrees test accuracy
Accuracy = 0.623675
Coverage = 1.000000
Overlap = 0.802120
defragTrees benchmark
Working on defragTrees for instance 1752
Working on defragTrees for instance 1202
Working on defragTrees for instance 1728
Working on defragTrees for instance 512
Working on defragTrees for instance 827
Working on defragTrees for instance 637
Working on defragTrees for instance 266
Working on defragTrees for instance 1629
Working on defragTrees for instance 717
Working on defragTrees for instance 522
Working on defragTrees for instance 328
Working on defragTrees for instance 1518
Working on defragTrees for instance 746
Working on defragTrees for insta

In [10]:
x = [[12, 12, 34, 4], [13, 4, 555, 4], [213, 4,4,2]]

In [11]:
np.sum(x, axis=0)

array([238,  20, 593,  10])

In [12]:
np.mean(np.array([-8.73381952 , 8.60892032 , 0.41378553, -0.28888634]) + 20)

19.999999997499998